In [1]:
import pandas as pd
import numpy as np

from scipy.stats import spearmanr as scor

In [80]:
ctd2=pd.read_csv('../results/consensus_conc.csv',sep=',',header=0,index_col=0)

In [81]:
sig_rep=pd.read_csv('../data/external/rep_consensus_signatures.csv',sep=',',header=0,index_col=0)
sig_ctrp=pd.read_csv('../data/external/ctrp_consensus_signatures.csv',sep=',',header=0,index_col=0)
sig_shrna=pd.read_csv('../data/external/shrna_consensus_signatures.csv',sep=',',header=0,index_col=0)

In [82]:
gene_info=pd.read_csv('../data/external/LINCS/GSE92742_Broad_LINCS_gene_info.txt',sep='\t',header=0,index_col=0)
fil=gene_info['pr_is_lm']==1
gene_info=gene_info[fil]
gene_info.index=gene_info.index.astype(str)

In [83]:
sig_rep.columns=gene_info.loc[sig_rep.columns]['pr_gene_symbol']
sig_ctrp.columns=gene_info.loc[sig_ctrp.columns]['pr_gene_symbol']
sig_shrna=sig_shrna[sig_ctrp.columns]

In [84]:
genes=list(set(ctd2.index) & set(sig_ctrp.columns))
ctd2=ctd2.loc[genes]
sig_rep=sig_rep[genes].T
sig_ctrp=sig_ctrp[genes].T
sig_shrna=sig_shrna[genes].T

In [85]:
sig_rep_all=pd.concat([sig_rep,ctd2],1)
sig_ctrp_all=pd.concat([sig_ctrp,ctd2],1)
sig_shrna_all=pd.concat([sig_shrna,ctd2],1)

In [86]:
sim_rep=pd.DataFrame(scor(sig_rep_all)[0],
                     index=sig_rep_all.columns,columns=sig_rep_all.columns).loc[sig_rep.columns,ctd2.columns]
sim_ctrp=pd.DataFrame(scor(sig_ctrp_all)[0],
                      index=sig_ctrp_all.columns,columns=sig_ctrp_all.columns).loc[sig_ctrp.columns,ctd2.columns]
sim_shrna=pd.DataFrame(scor(sig_shrna_all)[0],
                       index=sig_shrna_all.columns,columns=sig_shrna_all.columns).loc[sig_shrna.columns,ctd2.columns]

In [91]:
sim_rep.to_csv('../predictions/similarity/sim_rep.csv',sep=',')
sim_ctrp.to_csv('../predictions/similarity/sim_ctrp.csv',sep=',')

sim_shrna.to_csv('../predictions/similarity/sim_shrna.csv',sep=',')

In [100]:
data={'sim_rep':sim_rep,'sim_ctrp':sim_ctrp,'sim_shrna':sim_shrna}

In [101]:
for i in range(3):
    s=[0.01,0.03,0.1][i]
    for sim in data.keys():
        temp=data[sim].copy()
        temp+=np.random.normal(0,s,temp.shape)
        temp.to_csv('../predictions/similarity/%s_%s.csv' % (sim,str(i)),sep=',')

In [10]:
temp=pd.read_csv('../predictions/similarity/sim_rep.csv',sep=',',header=0,index_col=0)
temp+=np.random.normal(0,0.3,temp.shape)
temp.to_csv('../predictions/similarity/sim_rep_3.csv',sep=',')